In [1]:
import os
import re
import sys
import itertools
from pathlib import Path
from multiprocessing import Pool, cpu_count

from tqdm import tqdm
import numpy as np

from my_package.textfile2df import nnlist2df


In [6]:

# # どの距離のPOSCAR.nnlistを，CO3を含むPOSCARかどうかの判定に使うのかを受け取る
# args = sys.argv


def flatten_func(list_2dim):
    return list(itertools.chain.from_iterable(list_2dim))


def bool_CO3_contained_poscar(poscar_nnlist):
    df_nnlist = nnlist2df(str(poscar_nnlist))

    # df_nnlistでcentral speciesがCのものに絞る
    df_nnlist_central_species_C = df_nnlist[df_nnlist['central species'] == 'C']

    # さらに，炭酸イオンかどうかを判定するためにに，あるcentral atomのneighboring speciesがCとO３つの計４つでできているか確認したい
    # central atomの値を入力すれば，neighboring speciesのリストを返す関数()を作成
    def (central_atom_id, df=df_nnlist_central_species_C):
        """
        To get all central atoms of a cluster(:neighbors), Input a number of cluster center element number(:central atom)

        Input: central atom column element In df_nnlist
     -> Output: All neighboring atom column element that Input(:elemnt) match central atom column element

        param1: Input: central atom column element In df_nnlist
        """
        # 左側の列から対応する行を選択し、右側の数値を取得
        # result = df_nnlist[df_nnlist['central atom'] == input_value]['neighboring atom'].values
        neighboring_species_list = df[df['central atom'] == central_atom_id]['neighboring species'].tolist()
        return neighboring_species_list

    def match_num_C_O_3(central_atom_id):
        # 原子団の要素数は，Cが1つ，Oが3つの計4つかどうかcheck
        if len((central_atom_id)) == 4:
            # 原子団の要素にCが1つだけ含まれているかどうかcheck
            if (central_atom_id).count('C') == 1:
                # 原子団の要素にOがちょうど3つ含まれているかどうかcheck
                if (central_atom_id).count('O') == 3:
                    return True
            else:
                return False
        else:
            return False

    # C2O6のようなCO3に似通っているものを排除する
    def match_C_O_3():

        def has_duplicates(seq):
            return len(seq) != len(set(seq))
    
        # df_nnlist_central_species_Cに対し，CO3がどうかを確認し，CO3である原子のid一覧を取得
        # まず，中心元素がCのid一覧(central atomの値の一覧)を取得
        central_species_C_list = df_nnlist_central_species_C['central atom'].unique()
        
        
        species_id_list = flatten_func(list(map(, central_species_C_list)))
        set_species_id_list = list(set(species_id_list))
        
        for num in set_species_id_list:
            species_id_list.remove(num)
        
        
        central_species_C_dict = {}
        for i in central_species_C_list:    
            central_species_C_dict[i] = (i)
            
        CO3_filter_list = []
        for duplicated_num in species_id_list:
            CO3_filter_list.append([not (duplicated_num in list_) for list_ in central_species_C_dict.values()])
            #論理積
            CO3_filter = [min(t) for t in zip(*CO3_filter_list)]
        
        return CO3_filter
            
        
        
    match_C_O_3()
            
            
        
        # CO3_filter = [not (duplicated_num in i) for duplicated_num in species_id_list for i in central_species_C_dict.values()]
        
        
        
#         for duplicated_num in species_id_list:
#             CO3_filter = [not (duplicated_num in i) for i in central_species_C_dict.values()]
        
        
        
        
        
        
        
        
#         if has_duplicates(species_list):
            
#         else:
#             return True
        
        
        
    
    # df_nnlist_central_species_Cに対し，CO3がどうかを確認し，CO3である原子のid一覧を取得
    # まず，中心元素がCのid一覧(central atomの値の一覧)を取得
    central_species_C_list = df_nnlist_central_species_C['central atom'].unique()
    # その中で，match_num_C_O_3()を用いて，過不足なくCO3だけを含むものに絞る
    matched_central_species_C_list = [i for i in central_species_C_list if match_num_C_O_3(i)]

    return True if len(matched_central_species_C_list) > 0 else False



In [7]:
bool_CO3_contained_poscar('POSCAR.nnlist')

In [19]:
!head POSCAR.nnlist

      1     1    0.000000    0.000000   0.000000   0.000000    0   0   0  O O
      1   131    1.275056    0.650996   0.267192   1.063287    0   0   0  O C
      2     2    0.000000    0.000000   0.000000   0.000000    0   0   0  O O
      2   132    1.275056    0.650996  -0.267192   1.063287    0   0   0  O C
      3     3    0.000000    0.000000   0.000000   0.000000    0   0   0  O O
      3   133    1.275056   -0.650996  -0.267192  -1.063287    0   0   0  O C
      4     4    0.000000    0.000000   0.000000   0.000000    0   0   0  O O
      4   134    1.275056   -0.650996   0.267192  -1.063287    0   0   0  O C
      5     5    0.000000    0.000000   0.000000   0.000000    0   0   0  O O
      5   131    1.268835   -1.214450   0.367389  -0.008861    0   0   0  O C


In [40]:
# C2O6のようなCO3に似通っているものを排除する

poscar_nnlist = 'POSCAR.nnlist'
df_nnlist = nnlist2df(str(poscar_nnlist))

# df_nnlistでcentral speciesがCのものに絞る
df_nnlist_central_species_C = df_nnlist[df_nnlist['central species'] == 'C']

# さらに，炭酸イオンかどうかを判定するためにに，あるcentral atomのneighboring speciesがCとO３つの計４つでできているか確認したい
# central atomの値を入力すれば，neighboring speciesのリストを返す関数get_neighboring_species_list()を作成
def get_neighboring_species_list(central_atom_id, df=df_nnlist_central_species_C):
    """
    To get all central atoms of a cluster(:neighbors), Input a number of cluster center element number(:central atom)

    Input: central atom column element In df_nnlist
 -> Output: All neighboring atom column element that Input(:elemnt) match central atom column element

    param1: Input: central atom column element In df_nnlist
    """
    # 左側の列から対応する行を選択し、右側の数値を取得
    # result = df_nnlist[df_nnlist['central atom'] == input_value]['neighboring atom'].values
    neighboring_species_list = df[df['central atom'] == central_atom_id]['neighboring species'].tolist()
    return neighboring_species_list


def get_neighboring_atom_id_list(central_atom_id, df=df_nnlist_central_species_C):
    """
    To get all central atoms of a cluster(:neighbors), Input a number of cluster center element number(:central atom)

    Input: central atom column element In df_nnlist
 -> Output: All neighboring atom column element that Input(:elemnt) match central atom column element

    param1: Input: central atom column element In df_nnlist
    """
    # 左側の列から対応する行を選択し、右側の数値を取得
    # result = df_nnlist[df_nnlist['central atom'] == input_value]['neighboring atom'].values
    neighboring_atom_id_list = df[df['central atom'] == central_atom_id]['neighboring atom'].tolist()
    return neighboring_atom_id_list



    # def has_duplicates(seq):
    #     return len(seq) != len(set(seq))

# df_nnlist_central_species_Cに対し，CO3がどうかを確認し，CO3である原子のid一覧を取得
# まず，中心元素がCのid一覧(central atomの値の一覧)を取得
central_species_C_list = df_nnlist_central_species_C['central atom'].unique()
# import pdb; pdb.set_trace()

species_id_list = flatten_func(list(map(get_neighboring_atom_id_list, central_species_C_list)))
set_species_id_list = list(set(species_id_list))
# import pdb; pdb.set_trace()

for num in set_species_id_list:
    species_id_list.remove(num)


central_species_C_dict = {}
for i in central_species_C_list:    
    central_species_C_dict[i] = get_neighboring_atom_id_list(i)
# import pdb; pdb.set_trace()
    


# import pdb; pdb.set_trace()

# for duplicated_num in species_id_list:
#     CO3_filter_list.append([not (duplicated_num in list_) for list_ in central_species_C_dict.values()])

    

    
# import pdb; pdb.set_trace()
    #論理積
    # CO3_filter = [min(t) for t in zip(*CO3_filter_list)]
   

# CO3_filter
# import pdb; pdb.set_trace()


In [41]:
central_species_C_dict

{131: [1, 5, 9, 131],
 132: [2, 6, 10, 132],
 133: [3, 7, 11, 133],
 134: [4, 8, 12, 134],
 135: [13, 17, 135],
 136: [14, 18, 136],
 137: [15, 19, 137],
 138: [16, 20, 138],
 139: [27, 29, 139],
 140: [28, 30, 140],
 141: [25, 31, 141],
 142: [26, 32, 142],
 143: [37, 41, 143],
 144: [38, 42, 144],
 145: [39, 43, 145],
 146: [40, 44, 146],
 147: [49, 147],
 148: [50, 148],
 149: [57, 149],
 150: [58, 150],
 151: [55, 151],
 152: [56, 152],
 153: [67, 153],
 154: [68, 154]}

In [43]:
[list_ for list_ in central_species_C_dict.values()]

[[1, 5, 9, 131],
 [2, 6, 10, 132],
 [3, 7, 11, 133],
 [4, 8, 12, 134],
 [13, 17, 135],
 [14, 18, 136],
 [15, 19, 137],
 [16, 20, 138],
 [27, 29, 139],
 [28, 30, 140],
 [25, 31, 141],
 [26, 32, 142],
 [37, 41, 143],
 [38, 42, 144],
 [39, 43, 145],
 [40, 44, 146],
 [49, 147],
 [50, 148],
 [57, 149],
 [58, 150],
 [55, 151],
 [56, 152],
 [67, 153],
 [68, 154]]

In [51]:
species_id_list = [131, 132, 134]

In [52]:
CO3_filter_list = []
for duplicated_num in species_id_list:
    CO3_filter_list.append([not (duplicated_num in list_) for list_ in central_species_C_dict.values()])


In [53]:
CO3_filter_list

[[False,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True],
 [True,
  False,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True],
 [True,
  True,
  True,
  False,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True]]

In [57]:

# 論理積
CO3_filter = [min(t) for t in zip(*CO3_filter_list)]    
CO3_filter

[False,
 False,
 True,
 False,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True]

In [63]:
CO3_filter = np.array(CO3_filter)
CO3_filter = ~CO3_filter

In [64]:
CO3_filter

array([ True,  True, False,  True, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False])

24

## コード成形

In [134]:
# C2O6のようなCO3に似通っているものを排除する

poscar_nnlist = 'POSCAR.nnlist'
df_nnlist = nnlist2df(str(poscar_nnlist))

# df_nnlistでcentral speciesがCのものに絞る
df_nnlist_central_species_C = df_nnlist[df_nnlist['central species'] == 'C']

# さらに，炭酸イオンかどうかを判定するためにに，あるcentral atomのneighboring speciesがCとO３つの計４つでできているか確認したい
# central atomの値を入力すれば，neighboring speciesのリストを返す関数get_neighboring_species_list()を作成
def get_neighboring_species_list(central_atom_id, df=df_nnlist_central_species_C):
    """
    To get all central atoms of a cluster(:neighbors), Input a number of cluster center element number(:central atom)

    Input: central atom column element In df_nnlist
 -> Output: All neighboring atom column element that Input(:elemnt) match central atom column element

    param1: Input: central atom column element In df_nnlist
    """
    # 左側の列から対応する行を選択し、右側の数値を取得
    # result = df_nnlist[df_nnlist['central atom'] == input_value]['neighboring atom'].values
    neighboring_species_list = df[df['central atom'] == central_atom_id]['neighboring species'].tolist()
    return neighboring_species_list


def get_neighboring_atom_id_list(central_atom_id, df=df_nnlist_central_species_C):
    """
    To get all central atoms of a cluster(:neighbors), Input a number of cluster center element number(:central atom)

    Input: central atom column element In df_nnlist
 -> Output: All neighboring atom column element that Input(:elemnt) match central atom column element

    param1: Input: central atom column element In df_nnlist
    """
    # 左側の列から対応する行を選択し、右側の数値を取得
    # result = df_nnlist[df_nnlist['central atom'] == input_value]['neighboring atom'].values
    neighboring_atom_id_list = df[df['central atom'] == central_atom_id]['neighboring atom'].tolist()
    return neighboring_atom_id_list



    # def has_duplicates(seq):
    #     return len(seq) != len(set(seq))

# df_nnlist_central_species_Cに対し，CO3がどうかを確認し，CO3である原子のid一覧を取得
# まず，中心元素がCのid一覧(central atomの値の一覧)を取得
central_species_C_list = df_nnlist_central_species_C['central atom'].unique()
import pdb; pdb.set_trace()

species_id_list = flatten_func(list(map(get_neighboring_atom_id_list, central_species_C_list)))
set_species_id_list = list(set(species_id_list))
# import pdb; pdb.set_trace()

for num in set_species_id_list:
    species_id_list.remove(num)


central_species_C_dict = {}
for i in central_species_C_list:    
    central_species_C_dict[i] = get_neighboring_atom_id_list(i)
# import pdb; pdb.set_trace()
    


# import pdb; pdb.set_trace()

for duplicated_num in species_id_list:
    CO3_filter_list.append([not (duplicated_num in list_) for list_ in central_species_C_dict.values()])

# 論理積
CO3_filter = [min(t) for t in zip(*CO3_filter_list)]
   
CO3_matched_central_species_C_list = np.array(list(central_species_C_dict.keys()))[CO3_filter]
# CO3_filter
# import pdb; pdb.set_trace()


--Return--
None
> /tmp/ipykernel_1465283/3828249376.py(48)<module>()
     46 # まず，中心元素がCのid一覧(central atomの値の一覧)を取得
     47 central_species_C_list = df_nnlist_central_species_C['central atom'].unique()
---> 48 import pdb; pdb.set_trace()
     49 
     50 species_id_list = flatten_func(list(map(get_neighboring_atom_id_list, central_species_C_list)))



ipdb>  exit


In [70]:
CO3_matched_central_species_C_list

array([133, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146,
       147, 148, 149, 150, 151, 152, 153, 154])

## 整形＆関数化

In [89]:
def flatten_func(list_2dim):
    return list(itertools.chain.from_iterable(list_2dim))


def bool_CO3_contained_poscar(poscar_nnlist):
    df_nnlist = nnlist2df(str(poscar_nnlist))

    # df_nnlistでcentral speciesがCのものに絞る
    df_nnlist_central_species_C = df_nnlist[df_nnlist['central species'] == 'C']

    # さらに，炭酸イオンかどうかを判定するためにに，あるcentral atomのneighboring speciesがCとO３つの計４つでできているか確認したい
    # central atomの値を入力すれば，neighboring speciesのリストを返す関数()を作成
    def get_neighboring_species_list(central_atom_id, df=df_nnlist_central_species_C):
        """
        To get all central atoms of a cluster(:neighbors), Input a number of cluster center element number(:central atom)

        Input: central atom column element In df_nnlist
     -> Output: All neighboring atom column element that Input(:elemnt) match central atom column element

        param1: Input: central atom column element In df_nnlist
        """
        # 左側の列から対応する行を選択し、右側の数値を取得
        # result = df_nnlist[df_nnlist['central atom'] == input_value]['neighboring atom'].values
        neighboring_species_list = df[df['central atom'] == central_atom_id]['neighboring species'].tolist()
        return neighboring_species_list

    def match_num_C_O_3(central_atom_id):
        # 原子団の要素数は，Cが1つ，Oが3つの計4つかどうかcheck
        if len(get_neighboring_species_list(central_atom_id)) == 4:
            # 原子団の要素にCが1つだけ含まれているかどうかcheck
            if get_neighboring_species_list(central_atom_id).count('C') == 1:
                # 原子団の要素にOがちょうど3つ含まれているかどうかcheck
                if get_neighboring_species_list(central_atom_id).count('O') == 3:
                    return True
            else:
                return False
        else:
            return False
    
    # df_nnlist_central_species_Cに対し，CO3がどうかを確認し，CO3である原子のid一覧を取得
    # まず，中心元素がCのid一覧(central atomの値の一覧)を取得
    central_species_C_id_list = df_nnlist_central_species_C['central atom'].unique()
    # その中で，match_num_C_O_3()を用いて，過不足なくCO3だけを含むものに絞る
    num_matched_central_species_C_id_list = [i for i in central_species_C_id_list if match_num_C_O_3(i)]
    
    
    def rm_duplicated_central_species_C_id(central_species_C_list=num_matched_central_species_C_id_list):
        def get_neighboring_atom_id_list(central_atom_id, df=df_nnlist_central_species_C):
            """
            To get all central atoms of a cluster(:neighbors), Input a number of cluster center element number(:central atom)

            Input: central atom column element In df_nnlist
         -> Output: All neighboring atom column element that Input(:elemnt) match central atom column element

            param1: Input: central atom column element In df_nnlist
            """
            # 左側の列から対応する行を選択し、右側の数値を取得
            # result = df_nnlist[df_nnlist['central atom'] == input_value]['neighboring atom'].values
            neighboring_atom_id_list = df[df['central atom'] == central_atom_id]['neighboring atom'].tolist()
            return neighboring_atom_id_list

        # 中心がC，その周りにOが3つ存在する原子のid一覧をリスト化
        species_id_list = flatten_func(list(map(get_neighboring_atom_id_list, central_species_C_list)))
        # 直上のspecies_id_listの重複削除してリスト化
        set_species_id_list = list(set(species_id_list))
        # species_id_listの要素から，set_species_id_listの要素を削除 -> species_id_listにはCO3ではない原子のリストが入る
        for num in set_species_id_list:
            species_id_list.remove(num)

        # central_atom_idをkey，そのneighboring atom idをvalueにして，CO3が塊まった形でデータ保持する
        central_species_C_dict = {}
        for i in central_species_C_list:    
            central_species_C_dict[i] = get_neighboring_atom_id_list(i)

        # species_id_listにはCO3ではない原子のリストが入っている
        # CO3でない原子のidを含むCO3の塊はFalse, それ以外のCO3はTureを返し，リスト化
        CO3_filter_list = []
        for duplicated_num in species_id_list:
            CO3_filter_list.append([not (duplicated_num in value_list) for value_list in central_species_C_dict.values()])

        # 2重リストになっているCO3_filter_listを要素ごとに論理積（ex). [[True, False, True], [False, True, Ture]] -> [False, False, True]）
        CO3_filter = [min(t) for t in zip(*CO3_filter_list)]
        # match_C_O_3で絞ったcentral_species_C_listから，さらに重複してCO3になっているものを除去（ex). C2O6）
        CO3_matched_central_species_C_list = np.array(list(central_species_C_dict.keys()))[CO3_filter]

    return True if len(CO3_matched_central_species_C_list) > 0 else False

In [90]:
bool_CO3_contained_poscar('POSCAR.nnlist')

True

In [168]:
def bool_CO3_contained_poscar(poscar_nnlist):
    df_nnlist = nnlist2df(str(poscar_nnlist))

    # df_nnlistでcentral speciesがCのものに絞る
    df_nnlist_central_species_C = df_nnlist[df_nnlist['central species'] == 'C']

    # さらに，炭酸イオンかどうかを判定するためにに，あるcentral atomのneighboring speciesがCとO３つの計４つでできているか確認したい
    # central atomの値を入力すれば，neighboring speciesのリストを返す関数()を作成
    def get_neighboring_species_list(central_atom_id, df=df_nnlist_central_species_C):
        """
        To get all central atoms of a cluster(:neighbors), Input a number of cluster center element number(:central atom)

        Input: central atom column element In df_nnlist
     -> Output: All neighboring atom column element that Input(:elemnt) match central atom column element

        param1: Input: central atom column element In df_nnlist
        """
        # 左側の列から対応する行を選択し、右側の数値を取得
        # result = df_nnlist[df_nnlist['central atom'] == input_value]['neighboring atom'].values
        neighboring_species_list = df[df['central atom'] == central_atom_id]['neighboring species'].tolist()
        return neighboring_species_list

    def match_num_C_O_3(central_atom_id):
        # 原子団の要素数は，Cが1つ，Oが3つの計4つかどうかcheck
        if len(get_neighboring_species_list(central_atom_id)) == 4:
            # 原子団の要素にCが1つだけ含まれているかどうかcheck
            if get_neighboring_species_list(central_atom_id).count('C') == 1:
                # 原子団の要素にOがちょうど3つ含まれているかどうかcheck
                if get_neighboring_species_list(central_atom_id).count('O') == 3:
                    return True
            else:
                return False
        else:
            return False
    
    # df_nnlist_central_species_Cに対し，CO3がどうかを確認し，CO3である原子のid一覧を取得
    # まず，中心元素がCのid一覧(central atomの値の一覧)を取得
    central_species_C_id_list = df_nnlist_central_species_C['central atom'].unique()
    # その中で，match_num_C_O_3()を用いて，過不足なくCO3だけを含むものに絞る
    num_matched_central_species_C_id_list = [i for i in central_species_C_id_list if match_num_C_O_3(i)]
    
    
    def rm_duplicated_central_species_C_id(central_species_C_list=num_matched_central_species_C_id_list):
        def get_neighboring_atom_id_list(central_atom_id, df=df_nnlist_central_species_C):
            """
            To get all central atoms of a cluster(:neighbors), Input a number of cluster center element number(:central atom)

            Input: central atom column element In df_nnlist
         -> Output: All neighboring atom column element that Input(:elemnt) match central atom column element

            param1: Input: central atom column element In df_nnlist
            """
            # 左側の列から対応する行を選択し、右側の数値を取得
            # result = df_nnlist[df_nnlist['central atom'] == input_value]['neighboring atom'].values
            neighboring_atom_id_list = df[df['central atom'] == central_atom_id]['neighboring atom'].tolist()
            return neighboring_atom_id_list

        # 中心がC，その周りにOが3つ存在する原子のid一覧をリスト化
        species_id_list = flatten_func(list(map(get_neighboring_atom_id_list, num_matched_central_species_C_id_list)))
        print(f"species_id_list: {species_id_list}")
        # 直上のspecies_id_listの重複削除してリスト化
        set_species_id_list = list(set(species_id_list))
        print(f"set_species_id_list: {set_species_id_list}")
        # species_id_listの要素から，set_species_id_listの要素を削除 -> species_id_listにはCO3ではない原子のリストが入る
        for num in set_species_id_list:
            species_id_list.remove(num)
        ###### TEST #####
        species_id_list.append(5)
        species_id_list.append(6)
        ###### TEST #####
        print(f"species_id_list: {species_id_list}")
        # species_id_listの要素数が0 -> Oの重複抽出なし -> すべてCO3の炭酸イオン（：C2O6でない）
        if len(species_id_list) == 0:
            return True
        else:
            # central_atom_idをkey，そのneighboring atom idをvalueにして，CO3が塊まった形でデータ保持する
            central_species_C_dict = {}
            for i in central_species_C_list:    
                central_species_C_dict[i] = get_neighboring_atom_id_list(i)
            print(f"central_species_C_dict: {central_species_C_dict}")
            # species_id_listにはCO3ではない原子のリストが入っている
            # CO3でない原子のidを含むCO3の塊はFalse, それ以外のCO3はTureを返し，リスト化
            CO3_filter_list = []
            for duplicated_num in species_id_list:
                print(f"duplicated_num: {duplicated_num}")
                CO3_filter_list.append([not (duplicated_num in value_list) for value_list in central_species_C_dict.values()])
            print(f"CO3_filter_list: {CO3_filter_list}")
            # 2重リストになっているCO3_filter_listを要素ごとに論理積（ex). [[True, False, True], [False, True, Ture]] -> [False, False, True]）
            CO3_filter = [min(t) for t in zip(*CO3_filter_list)]
            # match_C_O_3で絞ったcentral_species_C_listから，さらに重複してCO3になっているものを除去（ex). C2O6）
            CO3_matched_central_species_C_list = np.array(list(central_species_C_dict.keys()))[CO3_filter]

            return True if len(CO3_matched_central_species_C_list) > 0 else False

    return rm_duplicated_central_species_C_id()

In [169]:
bool_CO3_contained_poscar('POSCAR.nnlist')

species_id_list: [1, 5, 7, 16, 2, 6, 8, 15]
set_species_id_list: [1, 2, 5, 6, 7, 8, 15, 16]
species_id_list: [5, 6]
central_species_C_dict: {1: [1, 5, 7, 16], 2: [2, 6, 8, 15]}
duplicated_num: 5
duplicated_num: 6
CO3_filter_list: [[False, True], [True, False]]


False

In [163]:
for i in []:
    print(i)

In [ ]:
for duplicated_num in species_id_list:
        print(f"duplicated_num: {duplicated_num}")
        CO3_filter_list.append([not (dupli

In [156]:
[] in [1, 5]

False

In [120]:
df_nnlist = nnlist2df(str('POSCAR.nnlist'))

In [125]:
df_nnlist['central atom'] == 'C'

0     False
1     False
2     False
3     False
4     False
5     False
6     False
7     False
8     False
9     False
10    False
11    False
12    False
13    False
14    False
15    False
16    False
17    False
18    False
19    False
20    False
21    False
22    False
23    False
24    False
25    False
26    False
27    False
28    False
29    False
30    False
31    False
32    False
33    False
34    False
35    False
36    False
37    False
Name: central atom, dtype: bool

In [128]:
df_nnlist_central_species_C = df_nnlist[df_nnlist['central species'] == 'C']

In [131]:
df_nnlist_central_species_C['central atom'].unique()

array([1, 2])